# Keras Tuner

Keras Tuner es una biblioteca que te ayuda a elegir el conjunto óptimo de hiperparámetros para tu programa TensorFlow/Keras. El proceso de seleccionar el conjunto correcto de hiperparámetros para su aplicación de aprendizaje automático (ML) se llama _hyperparameter tuning_ o _hypertuning_.

Los hiperparámetros son las variables que gobiernan el proceso de entrenamiento y la topología de un modelo de Machine Learning. Estas variables permanecen constantes durante el proceso de entrenamiento e impactan directamente en el rendimiento del programa. Los hiperparámetros puden ser de dos tipos:

* **Hiperparámetros del modelo** que influyen en la selección del modelo, como el número y el ancho de las _hidden layers_
* **Hiperparámetros del algoritmo** que influyen en la velocidad y la calidad del algoritmo de aprendizaje, como la tasa de aprendizaje para el Descenso de Gradiente Estocástico (SGD). 

## 0. Imports

In [ ]:
import tensorflow as tf
from tensorflow import keras
import kerastuner as kt

## 1. Conjunto de datos

In [ ]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 4))
for index, e in enumerate(img_train[:8]):
    plt.subplot(1, 8, index+1)
    plt.imshow(e, cmap=plt.cm.gray)
    plt.title('Ejemplo: ' + str(index+1))
plt.show()

In [ ]:
# Normalizamos el valor de los pixels entre 0 y 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## 2. Definición del modelo

Cuando se construye un modelo para realizar _hypertuning_, debe definirse el espacio de búsqueda de hiperparámetros además de la arquitectura del modelo. El modelo que estableces para realizar _hypertuning_ se llama _hypermodel_.

Puedes definir un _hypermodel_ a través de dos enfoques:

* Implementando una función que construye el modelo
* Creando una subclase la clase `HyperModel` de la Keras Tuner API

También puedes utilizar dos clases predefinidas de `HyperModel`, `HyperXception` y `HyperResNet` para aplicaciones de visión artificial.

En este tutorial, se implementa una función que construye un modelo de clasificación de imágenes. La función devuelve un modelo compilado y utiliza los hiperparámetros que se definan para realizar _hypertuning_ del mismo.

In [ ]:
def model_builder(hp):
    # Definición del modelo
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28))) # una matriz 2D a 1D
    
    # Tunning del número de neuronasv('units') de la primera hidden layer
    # Seleccionamos el valor optimo entre 32 y 512 y él hace las pruebas por mi. de 32 en 32 neuronas
    hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
    model.add(keras.layers.Dense(units = hp_units, activation='relu'))
    
    # Tunning del ratio de aprendizaje para el algoritmo de optimizacion
    # Seleccionamos el valor optimo entre [0.01, 0.001, 0.0001]
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate = hp_learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'])
    return model

El parámetro `hp` que recibe la función, se corresponde con un objeto de la clase `HyperParameters`, que, de manera general, se corresponde con un "almacén" de hiperparámetros.

## 3. Instanciando el objeto Tuner para realizar _hypertuning_

Una vez definida la función anterior, se instancia el Tuner para realizar el _hypertuning_. El _Keras Tuner_ tiene cuatro _tuners_ disponibles - `RandomSearch`, `Hyperband`, `BayesianOptimization` y `Sklearn`. En este tutorial, se utiliza `Hyperband` (https://arxiv.org/pdf/1603.06560.pdf)

Para instanciar el _tuner_ de Hyperband, hay que especificar el _hypermodel_, el objetivo a optimizar y el número máximo de epochs a entrenar (max_epochs).

In [ ]:
tuner = kt.Hyperband(
    model_builder,
    objective = 'val_accuracy', 
    max_epochs = 10,
    factor = 3,
    directory = 'test_dir',
    project_name = 'hp_tuning'
)

El algoritmo `Hyperband` utiliza la asignación adaptativa de recursos y la detención temprana para converger rápidamente en un modelo de alto rendimiento. El algoritmo entrena un gran número de modelos durante unos pocos epochs y lleva a la siguiente ronda sólo la mitad de los modelos de mayor rendimiento. Hyperband determina el número de modelos a entrenar mediante la fórmula $1 + log_{factor}(max\_epochs)$ y redondea el resultado al entero más cercano.

In [ ]:
# Ejecución del tuner para la búsqueda (search) de hiperparámetros
tuner.search(
    img_train, 
    label_train, 
    epochs = 10, 
    validation_data = (img_test, label_test)
)

Una vez que hemos realizado la búsqueda de los hiperparámetros podemos acceder a los valores óptimos mediante la siugiente sentencia

In [ ]:
# Obtención de los hiperparámetros óptimos
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Units:", best_hps.get('units'))
print("Learning rate:", best_hps.get('learning_rate'))

## 5. Entrenamiento del modelo

Una vez seleccionados los hiperparámetros óptimos, los utilizamos para entrenar el modelo.

In [ ]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(
    img_train,
    label_train,
    epochs = 10,
    validation_data = (img_test, label_test)
)

In [ ]:
import pandas as pd
pd.DataFrame(history.history)[['accuracy', 'val_accuracy']].plot(figsize=(10, 7))
plt.grid(True)
plt.xlabel("epochs")
plt.show()